# 项目复现主流程 (Project Replication Main Pipeline)

本 Notebook 将作为项目的主控制台，引导您按顺序执行 `src/` 文件夹中的各个核心脚本，完成从数据准备到关键词发现的整个端到端流程。

**运行前请注意:**
* 请确保您已经安装了 `requirements.txt` 中的所有库 (`pip install -r requirements.txt`)。
* 请确保您是从项目的根目录 (`climate_keyword_discovery/`) 启动 Jupyter Notebook/Lab。
* 每个代码单元格对应算法的一个主要步骤。请按顺序执行。

### 准备工作：检查路径和数据

在开始前，我们先定义好所有需要的路径，并快速检查一下原始数据是否就位。

In [1]:
import os

# --- 定义项目核心路径 ---
RAW_DATA_PATH = os.path.join('data', 'raw')
PROCESSED_DATA_PATH = os.path.join('data', 'processed')
MODELS_PATH = 'models'
RESULTS_PATH = 'results'
SRC_PATH = 'src'

# --- 检查关键文件是否存在 ---
seed_file = os.path.join(RAW_DATA_PATH, 'initial_seed_bigrams.csv')

if not os.path.exists(seed_file):
    print(f"❌ 错误: 找不到种子词组文件，请确保 '{seed_file}' 文件存在！")
else:
    print(f"✅ 准备工作检查通过: 种子词组文件位于 '{seed_file}'")

print("\n项目路径设置完毕，可以开始执行流程。")

✅ 准备工作检查通过: 种子词组文件位于 'data/raw/initial_seed_bigrams.csv'

项目路径设置完毕，可以开始执行流程。


---
## 步骤 1: 构建参照集 (R) 和搜索集 (S)

我们将执行 `src/set_builder.py` 脚本。

该脚本会：
1.  加载 `data/raw/initial_seed_bigrams.csv` 中的种子词组。
2.  遍历 `data/raw/` 目录下的所有 `.txt` 文件并进行预处理。
3.  根据句子是否包含种子词组，将其分别归入参照集 (R) 和搜索集 (S)。
4.  将结果保存到 `data/processed/` 目录下。

In [1]:
print("--- 开始执行步骤 1: 构建 R 集和 S 集 ---")
%run src/set_builder.py
print("\n--- 步骤 1 执行完毕 ---")

--- 开始执行步骤 1: 构建 R 集和 S 集 ---
Loaded 50 seed bigrams.
Preprocessing file: data/raw/non_climate_texts/fiction_excerpt.txt
Preprocessing file: data/raw/non_climate_texts/local_news_article.txt
Preprocessing file: data/raw/transcripts/autotech_q3_2025.txt
Preprocessing file: data/raw/transcripts/energy_co_q3_2025.txt
Preprocessing file: data/raw/ipcc_reports/chapter_3_physical_basis.txt
Preprocessing file: data/raw/ipcc_reports/chapter_5_mitigation_pathways.txt

--- Set Building Complete ---
Reference Set (R) contains: 17 sentences.
Search Set (S) contains: 50 sentences.
Reference Set (R) saved to: data/processed/reference_set_R.json
Search Set (S) saved to: data/processed/search_set_S.json

--- 步骤 1 执行完毕 ---


---
## 步骤 2: 训练机器学习模型

现在，我们将执行 `src/model_trainer.py`。

该脚本会：
1.  加载上一步生成的 R 集和 S 集。
2.  构建训练集（R 作为正样本，S 的抽样作为负样本）。
3.  使用 TF-IDF 将文本向量化。
4.  通过网格搜索交叉验证来训练和调优三个分类器（朴素贝叶斯、SVC、随机森林）。
5.  将训练好的最佳模型保存到 `models/` 目录下。

In [2]:
print("--- 开始执行步骤 2: 训练模型 (这可能需要一些时间) ---")
%run src/model_trainer.py
print("\n--- 步骤 2 执行完毕 ---")

--- 开始执行步骤 2: 训练模型 (这可能需要一些时间) ---
--- Loading Data ---
Loaded 17 sentences for Reference Set (R).
Loaded 50 sentences for Search Set (S).

--- Preparing Training Data ---
Created training set with 67 samples.
Vectorizing text data with TF-IDF...
Vectorizer saved.

--- Training NaiveBayes ---
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Best parameters for NaiveBayes: {'alpha': 0.5}
NaiveBayes model saved to: models/naivebayes_classifier.joblib

--- Training SVC ---
Fitting 3 folds for each of 32 candidates, totalling 96 fits
Best parameters for SVC: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
SVC model saved to: models/svc_classifier.joblib

--- Training RandomForest ---
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters for RandomForest: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
RandomForest model saved to: models/randomforest_classifier.joblib

--- 步骤 2 执行完毕 ---


**结果验证:**

这个过程会花费一些时间。运行结束后，您应该会在 `models` 目录下看到四个 `.joblib` 文件：
* `tfidf_vectorizer.joblib`
* `naivebayes_classifier.joblib`
* `svc_classifier.joblib`
* `randomforest_classifier.joblib`

---
## 步骤 3: 预测并生成目标集 (T)

接下来，执行 `src/predictor.py`。

该脚本会：
1.  加载上一步训练好的模型和向量化器。
2.  使用模型对搜索集 (S) 中的每一个句子进行概率预测。
3.  将任何一个模型预测概率超过阈值（我们在脚本中设为0.5）的句子筛选出来，构成目标集 (T)。
4.  将结果保存到 `data/processed/` 目录下。

In [11]:
print("--- 开始执行步骤 3: 预测并生成 T 集 ---")
%run src/predictor.py
print("\n--- 步骤 3 执行完毕 ---")

--- 开始执行步骤 3: 预测并生成 T 集 ---
--- Loading Models and Vectorizer ---
Successfully loaded vectorizer and all 3 models.

--- Loading Search Set (S) ---
Scanning 50 sentences in the Search Set (S)...

--- Prediction Complete ---
Found 9 sentences for the Target Set (T).
Target Set (T) saved to: data/processed/target_set_T.json

--- 步骤 3 执行完毕 ---


**结果验证:**

运行结束后，在 `data/processed` 目录下，您会看到新生成的 `target_set_T.json` 文件。

---
## 步骤 4: 发现新关键词并生成最终词库

这是流程的最后一步，我们将执行 `src/keyword_discoverer.py`。

该脚本会：
1.  加载 T 集、S 集和初始种子词组。
2.  通过比较词组在 T 集和非 T 集中的文档频率，来发现新的关键词。
3.  将新发现的关键词与初始种子词组合并，形成最终词库。
4.  将最终词库保存为 `final_bigram_library.csv`。

In [3]:
print("--- 开始执行步骤 4: 发现新关键词 ---")
%run src/keyword_discovery_v2.py
print("\n--- 步骤 4 执行完毕 ---")

--- 开始执行步骤 4: 发现新关键词 ---
--- Loading T, S, and initial seed bigrams ---
Target Set (T) size: 9
Non-Target Set (S\T) size: 41

--- Calculating Document Frequencies ---
--- Calculating likelihood scores for candidate bigrams ---
Kept top 5% of candidates: 3 new bigrams.

--- Final Bigram Library Created ---
Total bigrams in the final library: 53
Final bigram library saved to: data/processed/final_bigram_library.csv

--- Example new bigrams (if any) ---
[('Kyoto', 'protocol'), ('air', 'pollution'), ('air', 'quality'), ('air', 'temperature'), ('biomass', 'energy'), ('carbon', 'dioxide'), ('carbon', 'emission'), ('carbon', 'energy'), ('carbon', 'neutral'), ('carbon', 'price')]

--- 步骤 4 执行完毕 ---


**结果验证:**

最终的成果 `final_bigram_library.csv` 会保存在 `data/processed` 目录下。您可以打开这个文件查看所有被发现的关键词。

---
## 结论

恭喜您！您已经成功地通过这个 Notebook 完整地运行了整个关键词发现算法的核心流程。

您现在拥有一个完整的、可以工作的关键词发现管道。接下来，您可以：
* 在 `notebooks` 文件夹下创建新的 Notebook，对 `final_bigram_library.csv` 进行分析。
* 编写 `src/exposure_calculator.py` 脚本来应用这个新词库，计算最终的风险敞口分数。
* 用更大规模、更真实的数据来替换 `data/raw` 中的文件，然后重新运行此 Notebook，看看能有什么新发现。